In [2]:
import time
import datetime
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import tushare as ts
ts.set_token('29eaf3bcac23df4c6d025de157ab2d53beead3391fbe6e83b4ebcb6c')
pro = ts.pro_api()

import mpl_finance as mpf
# import matplotlib.finance as mpf
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.pylab import date2num
import matplotlib as mpl
import seaborn as sns
sns.set()
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = 'SimHei'

In [3]:
# read 
from mylab.stock.myread import getIndexBasic # remote or local
from mylab.stock.myread import getIndexDaily
from mylab.stock.myread import getIndexWeekly
from mylab.stock.myread import getIndexMonthly
from mylab.stock.myread import getIndex
from mylab.stock.myread import getIndustryBasic  #local only
from mylab.stock.myread import getIndustryDaily
from mylab.stock.myread import getIndustryWeekly
from mylab.stock.myread import getIndustryMonthly
from mylab.stock.myread import getIndustry
# plot 
from mylab.stock.myplot import plotStock
from mylab.stock.myplot import displaySelect
# feature
from mylab.stock.myfeature import getMacd
from mylab.stock.myfeature import getMa
from mylab.stock.myfeature import getKdj
from mylab.stock.myfeature import extractFeature

# update weekly Industry Index

In [4]:
df = getIndexWeekly("399001.SZ", "20101001", "20200327",LOCAL = not True, market = "SZSE")
df.tail()

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount,trade_date_stamp
480,399001.SZ,20200228,10980.774,11620.631,11869.406,10962.242,11629.698,-648.924,-0.0558,1.442640e+11,2.098678e+12,737483.0
481,399001.SZ,20200306,11582.815,11115.459,11739.836,11053.288,10980.774,602.041,0.0548,1.202719e+11,1.734187e+12,737490.0
482,399001.SZ,20200313,10831.125,11363.794,11444.781,10380.171,11582.815,-751.690,-0.0649,1.106342e+11,1.515817e+12,737497.0
483,399001.SZ,20200320,10150.125,10867.662,10867.662,9719.492,10831.125,-681.000,-0.0629,9.710882e+10,1.367537e+12,737504.0
484,399001.SZ,20200327,10109.906,9828.898,10310.532,9634.967,10150.125,-40.219,-0.0040,8.114669e+10,1.097141e+12,737511.0


In [5]:
# glean all industry index
start_date = "201001001"
end_date = "20200430"
save_dir = "./data/industry/"
markets = ["SSE","SW","SZSE","OTH"] # markets

all_index_basic_df = pd.DataFrame()
for market in markets:
    for cat in ["一级行业指数","二级行业指数","三级行业指数","四级行业指数","行业指数"]:
        index_basic_df = getIndexBasic(LOCAL = not True, market = market )
        index_basic_df = index_basic_df.loc[(index_basic_df["category"] == cat), :] 
        if len(index_basic_df) == 0:
            continue
        print(market,cat, len(index_basic_df) )
        index_basic_df.reset_index(drop = True, inplace = True)
        all_index_basic_df = pd.concat([all_index_basic_df,index_basic_df],axis = 0)
        for i in range(len(index_basic_df)):
#             try:
            stock_code = index_basic_df.loc[i,"ts_code"]
            stock_name = index_basic_df.loc[i,"name"]
            index_df = getIndexDaily(stock_code, start_date, end_date,LOCAL = not True, market = market)
            index_df.to_csv(save_dir+"daily/"+ stock_code + stock_name + ".csv", index = False)
            index_df = getIndexWeekly(stock_code, start_date, end_date,LOCAL = not True, market = market)
            index_df.to_csv(save_dir+"weekly/"+ stock_code + stock_name + ".csv", index = False)
            index_df = getIndexMonthly(stock_code, start_date, end_date,LOCAL = not True, market = market)
            index_df.to_csv(save_dir+"monthly/"+ stock_code + stock_name + ".csv", index = False)
#             except:
#             print("err",stock_code,stock_name)
#                 pass
all_index_basic_df.reset_index(drop = True, inplace = True)
all_index_basic_df.to_csv(save_dir+ "all_industry_basic_info.csv", index = False)
print(all_index_basic_df.shape)
all_index_basic_df.head()

SSE 一级行业指数 87
SSE 二级行业指数 1
SSE 行业指数 4
SW 一级行业指数 34
SW 二级行业指数 113
SW 三级行业指数 252
SZSE 一级行业指数 61
SZSE 二级行业指数 8
SZSE 三级行业指数 1
SZSE 四级行业指数 1
OTH 一级行业指数 1
OTH 二级行业指数 14
(577, 8)


,ts_code,name,market,publisher,category,base_date,base_point,list_date
0,000032.SH,上证能源,SSE,中证公司,一级行业指数,20031231,1000.0,20090109
1,000033.SH,上证材料,SSE,中证公司,一级行业指数,20031231,1000.0,20090109
2,000034.SH,上证工业,SSE,中证公司,一级行业指数,20031231,1000.0,20090109
3,000035.SH,上证可选,SSE,中证公司,一级行业指数,20031231,1000.0,20090109
4,000036.SH,上证消费,SSE,中证公司,一级行业指数,20031231,1000.0,20090109


# select industry by KDJ

In [7]:
from mylab.stock.myselect import selectByKdj1

In [9]:
# select industry
start_date = "20140101"
end_date = "20200429"

industry_basic_df = getIndustryBasic()
print(industry_basic_df.shape)
all_select_df = pd.DataFrame()

for i in range(len(industry_basic_df)):
    stock_code = industry_basic_df.loc[i,"ts_code"]
    stock_name = industry_basic_df.loc[i,"name"]
    try:
        industry_df = getIndustry(stock_code,start_date,end_date,merge_daily = False)
    except:
        continue
    select_df = selectByKdj1(industry_df)
    if len(select_df) > 1:
        all_select_df = pd.concat([all_select_df,select_df],axis = 0)
    
all_select_df = all_select_df.loc[all_select_df["trade_date"] == end_date ,:]
all_select_df.to_csv("./data/industry/all_select_industry.csv",index = False)
all_select_df = pd.merge(all_select_df,industry_basic_df, how = "left", on = "ts_code")
all_select_df.reset_index(drop = True, inplace = True)
all_select_df.to_csv("./data/industry/all_select_industry.csv",index = False)
print(all_select_df.shape)
all_select_df.head()

(577, 8)
(0, 48)


,trade_date,close_weekly,open_weekly,high_weekly,low_weekly,pre_close_weekly,change_weekly,pct_chg_weekly,vol_weekly,amount_weekly,...,macd_monthly,macd_signal_monthly,ts_code,name,market,publisher,category,base_date,base_point,list_date


# plot index

In [7]:
# plot industry weekly
start_date = "20140101"
# end_date = "20200320"

all_select_df = pd.read_csv("./data/industry/all_select_industry.csv" )
for i in range(len(all_select_df)):
    stock_code = all_select_df.loc[i,"ts_code"]
    stock_name = all_select_df.loc[i,"name"]
    print(i,stock_code,stock_name)
    # plot weekly
    industry_df = getIndustryWeekly(stock_code, start_date, end_date)
    industry_df = getKdj(industry_df)
    industry_df = getMacd(industry_df)
    if len(industry_df) > 100:
        industry_df = industry_df.iloc[-100:,:]
    if len(industry_df)>10:
        plotStock(industry_df,SAVE = True ,save_dir = "./output/select_industry/"+stock_name+"-weekly-")
        plt.close()
     # plot monthly
    industry_df = getIndustryMonthly(stock_code, start_date, end_date)
    industry_df = getKdj(industry_df)
    industry_df = getMacd(industry_df)
    if len(industry_df) > 100:
        industry_df = industry_df.iloc[-100:,:]
    if len(industry_df)>10:
        plotStock(industry_df,SAVE = True ,save_dir = "./output/select_industry/"+stock_name+"-monthly-")
        plt.close()

0 801015.SI 渔业(申万)
1 801053.SI 黄金Ⅱ(申万)
2 801154.SI 医药商业Ⅱ(申万)
3 801155.SI 中药Ⅱ(申万)
4 801205.SI 商业物业经营(申万)
5 801211.SI 餐饮Ⅱ(申万)
6 801722.SI 装修装饰Ⅱ(申万)
7 850111.SI 种子生产(申万)
8 850313.SI 石油贸易(申万)
9 850363.SI 炭黑(申万)
10 850714.SI 内燃机(申万)
11 850912.SI 商用载货车(申万)
12 851521.SI 中药Ⅲ(申万)
13 851541.SI 医药商业Ⅲ(申万)
14 851631.SI 燃气Ⅲ(申万)
15 852031.SI 百货(申万)
16 852033.SI 多业态零售(申万)
17 852041.SI 专业连锁(申万)
18 852051.SI 一般物业经营(申万)
19 852112.SI 自然景点(申万)
20 852141.SI 餐饮Ⅲ(申万)
21 857221.SI 装修装饰Ⅲ(申万)
